<a href="https://colab.research.google.com/github/tylerb55/COMP530/blob/main/MobileNetV2Federated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/tylerb55/COMP530.git

Cloning into 'COMP530'...
remote: Enumerating objects: 4247, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 4247 (delta 14), reused 0 (delta 0), pack-reused 4216
Receiving objects: 100% (4247/4247), 917.09 MiB | 30.59 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [3]:
!pip install tqdm==4.28.1

     |████████████████████████████████| 45 kB 2.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.28.1 which is incompatible.
panel 0.12.1 requires tqdm>=4.48.0, but you have tqdm 4.28.1 which is incompatible.
fbprophet 0.7.1 requires tqdm>=4.36.1, but you have tqdm 4.28.1 which is incompatible.


In [4]:
! pip install tensorflow-federated==0.19.0

     |████████████████████████████████| 602 kB 7.5 MB/s 
     |████████████████████████████████| 887 kB 68.5 MB/s 
     |████████████████████████████████| 132 kB 81.0 MB/s 
     |████████████████████████████████| 14.8 MB 94.5 MB/s 
     |████████████████████████████████| 172 kB 94.7 MB/s 
     |████████████████████████████████| 4.0 MB 50.6 MB/s 
     |████████████████████████████████| 460.3 MB 7.2 kB/s 
     |████████████████████████████████| 192 kB 69.1 MB/s 
     |████████████████████████████████| 65.1 MB 167 kB/s 
     |████████████████████████████████| 1.2 MB 61.7 MB/s 
     |████████████████████████████████| 462 kB 74.2 MB/s 
  Created wheel for jax: filename=jax-0.2.28-py3-none-any.whl size=1028669 sha256=4577477dd8597ff97d71a17ac7f857d1e32aa8de221ffea9017bc3b56daede22
  Stored in directory: /root/.cache/pip/wheels/e2/60/4c/0cf931b766116b73950d9b6fca5813a45789d45d412a8d7272
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=37a56363b976160ffc

In [5]:
! pip install nest-asyncio

Import Necessary Libraries

In [6]:
import numpy as np
import pandas as pd
import collections
import matplotlib.image as img
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
import nest_asyncio as na
import tensorflow_hub as hub

#import keras

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8  # 0.6 sometimes works better for folks
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

ImportError: ignored

In [ ]:
#Have a play around with number of clients and batch size and see how they affect the model
NUM_CLIENTS=5 
NUM_EPOCHS=100
BATCH_SIZE=20
SHUFFLE_BUFFER=100
PREFETCH_BUFFER=10

In [ ]:
def Train_and_Test_Set(directory_name):
  """a function to load the images in a large directory into a tensorflow dataset object
  the data is split 80:10:10 in training:validation:test. The dataset is shuffled before splitting
  and the images are formatted to 512x512 pixels and grayscale(one color channel and the values range from 0 to 255)"""
  dataset_train=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    color_mode='rgb',
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.2,
                                                                    subset='training'
                                                                    )
  
  dataset_test=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    color_mode='rgb',
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.2,
                                                                    subset='validation'
                                                                    )
    
  return dataset_train,dataset_test

In [ ]:
def preprocess(dataset):

  def format_batch(element):
    """flatten the pixels in a batch and return the features as an 'OrderedDict'"""
    tf.image.per_image_standardization(element['image'])
    return collections.OrderedDict(
        x=tf.image.resize(element['image'],(224,224)),
        y=tf.reshape(element['label'],[1])
    )
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).map(format_batch).prefetch(PREFETCH_BUFFER)

In [ ]:
def federate_dataset(Dataset,clients):
  """
  args:
  Dataset - the dataset object to be passed into the function and separated between the clients in the simulation
  clients - the number of clients for the dataset to have come from in the simulated environment
  return:
  dataset_by_client - the original dataset federated between the number of clients. 
  each client is labelled by their client id, in a dictionary. each id points to an individual dataset object
  """
  image_count=tf.data.experimental.cardinality(Dataset).numpy()
  image_per_set=int(np.floor(image_count/clients))
  
  Dataset=Dataset.unbatch()
  client_train_dataset=collections.OrderedDict()
  Dataset=tfds.as_numpy(Dataset)

  count=0
  client_num=1
  """y=[]
  x=[]
  iterDataset=iter(Dataset)
  image_shape=next(iterDataset)[0].shape
  label_shape=next(iterDataset)[1].shape
  imagearr=np.empty(shape=image_shape)
  labelarr=np.empty(1)
  for image in Dataset:
    np.append(imagearr,image[0])
    np.append(labelarr,image[1])
    
  for i in range(1, clients+1):
    client_name = "client_" + str(i)
    start = image_per_set * (i-1)
    end = image_per_set * i

    print(f"Adding data from {start} to {end} for client : {client_name}")
    data = collections.OrderedDict((('label', labelarr[start:end]), ('pixels', imagearr[start:end])))
    client_train_dataset[client_name] = data

  federated_dataset=tff.simulation.datasets.TestClientData(client_train_dataset)
  return federated_dataset"""


  y=[]
  x=[]
  """this code snippet assigns particular training examples to each client. In this example they have been evenly distributed
  hetrogeneity of federated data could be explored by randomly assigning a clients to each training example.
  This would result in some clients having more than enough training data locally and some  and some suffering from data
  paucity locally. A greater reflection of real life struggles when implementing federated learning."""
  for image in Dataset:
    count+=1
    y.append(image[1])
    x.append(image[0])
    if(count==image_per_set):
      x=np.asarray(x,dtype=np.float32)
      y=np.asarray(y,dtype=np.int32)
      data=collections.OrderedDict((('label', y), ('image', x)))
      client_train_dataset["client_"+str(client_num)]=data
      count=0
      client_num+=1
      y=[]
      x=[]

  federated_dataset=tff.simulation.datasets.TestClientData(client_train_dataset)
  return federated_dataset


In [ ]:
def mobilenetv2_model(classes):
  return tf.keras.Sequential([
                              hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",input_shape=(224, 224, 3),trainable=False),
                              tf.keras.layers.Dense(classes,activation='sigmoid')
                              ])

In [ ]:
def make_federated_data(client_data,client_ids):
  return[
         preprocess(client_data.create_tf_dataset_for_client(x))
         for x in client_ids         
  ]

In [ ]:
""" the original dataset held in keras dataset objects """
dataset_train, dataset_test=Train_and_Test_Set("Dataset1")
""" the dataset split by the which client the data has come from """
federated_train_dataset=federate_dataset(dataset_train,NUM_CLIENTS)
federated_test_dataset=federate_dataset(dataset_test,NUM_CLIENTS)
""" an example dataset for a single client used to get the input specification for the federated model """
example_dataset = federated_train_dataset.create_tf_dataset_for_client(federated_train_dataset.client_ids[0])
preprocessed_example_dataset=preprocess(example_dataset)
""" the federated dataset that can be passed into the federated environemt to train the network """
federated_train_data = make_federated_data(federated_train_dataset, federated_train_dataset.client_ids)
federated_test_data = make_federated_data(federated_test_dataset, federated_test_dataset.client_ids)

In [ ]:
#model = mobilenetv2_model(1)
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#              metrics=['accuracy'])

In [ ]:
#def ResNetPreprocess(image,label):
#  image = image/255.0
  # Resize images from 512x512 to 224x224
#  image = tf.image.resize(image, (224,224))
#  return image, label

In [ ]:
#BATCH_SIZE=32

#dataset_train, dataset_test=Train_and_Test_Set("Dataset1")
#dataset_train=dataset_train.map(ResNetPreprocess)
#dataset_test=dataset_test.map(ResNetPreprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)

#AUTOTUNE = tf.data.AUTOTUNE
#dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#model.fit(dataset_train,epochs=3)

In [ ]:
def federated_model():
  MobileNetV2_Fed=mobilenetv2_model(1)
  return tff.learning.from_keras_model(
      MobileNetV2_Fed,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.Accuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
iterative_process = tff.learning.build_federated_averaging_process(
    federated_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [ ]:
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
na.apply()
state=iterative_process.initialize()
NUM_ROUNDS=11
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    print('round {:2d}, metrics={}'.format(round_num, metrics))
    for name, value in metrics['train'].items():
      tf.summary.scalar(name, value, step=round_num)

In [ ]:
!ls {logdir}
%load_ext tensorboard
%tensorboard --logdir {logdir} --port=0

Test the model on the test set and evaluate performance

In [ ]:
# Uncomment and run this cell to clean your directory of old output for
# future graphs from this directory. We don't run it by default so that if 
# you do a "Runtime > Run all" you don't lose your results.

# !rm -R /tmp/logs/scalars/*

Produce graphs and visualisations for evalutation data

In [ ]:
evaluation = tff.learning.build_federated_evaluation(federated_model)

In [ ]:
test_metrics = evaluation(state.model, federated_test_data)

In [ ]:
example_element = next(iter(example_dataset))

example_element['label'].numpy()

In [ ]:
federated_train_dataset.element_type_structure